Trying some things out. 
Mostly from here: https://www.kaggle.com/anokas/quora-question-pairs/data-analysis-xgboost-starter-0-35460-lb for now.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [ ]:
def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R


def similarity(q1, q2, use_tokenizer = False, remove_stop_words = True):
    #this does slightly worse than word_match
    """
    look at the jaccard distance between the sets of unique elements
    """
    if use_tokenizer:
        q1 = nltk.word_tokenize(q1.lower())
        q2 = nltk.word_tokenize(q2.lower())
    else:
        q1 = q1.lower().split()
        q2 = q2.lower().split()

    if remove_stop_words:
        q1 = [word for word in q1 if word not in stop_words]
        q2 = [word for word in q2 if word not in stop_words]


    n = len(np.union1d(q1, q1))
    if n != 0:
        sim = len(np.intersect1d(q1, q2))/n
    else:
        sim = 0
    return sim

In [ ]:
stops = stop_words = stopwords.words('english')

In [ ]:
df = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [ ]:
df = df.dropna()
#Rebalance the classes:
#pos_boostrap_sample = df[df["is_duplicate"] == 0].sample(n = 500000, replace = True)
#df = pd.concat((pos_boostrap_sample, df))

In [ ]:
df["word_share"] = df.apply(word_match_share, axis = 1, raw=True)

In [ ]:
df_test["word_share"] = df_test.apply(word_match_share, axis = 1, raw=True)

In [ ]:
X = df["word_share"].values.reshape(-1,1)
X_test = df_test["word_share"].values.reshape(-1,1)
y = df["is_duplicate"]

In [ ]:
model = RandomForestClassifier(n_estimators = 200, max_features = True)
model.fit(X, y)

In [ ]:
preds = model.predict_proba(X_test)[:,1]

In [ ]:
sub = pd.DataFrame()
sub['test_id'] = df_test['test_id']
sub['is_duplicate'] = preds
sub.to_csv('rf_pred.csv', index=False)

In [ ]:
sub.is_duplicate.hist()